# Assignment 4: Alarm System

In [1]:
from pynq.overlays.base import BaseOverlay
import multiprocessing
import os
import time
import socket
from datetime import datetime
base = BaseOverlay("base.bit")


### Interacting with GPIO from MicroBlaze to control Buzzer

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a Buzzer pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Fuction to reset buzzer GPIO pin on PMODB
void reset_buzzer(){
        write_gpio(2,0);
}

### Alarm Tone and Duration 

In [3]:
#Function to emulate Alarm tone. frequency is in Hz
def setAlarm(frequency):
    sleepVal = 1/(2*frequency)    # define sleep time for duty cycle
    end_time = time.time() + .5   # define alarm duration to be .5 seconds
    while time.time() <end_time:  # run alarm tone for .5 seconds
        write_gpio(2,1)
        time.sleep(sleepVal)
        write_gpio(2,0)
        time.sleep(sleepVal)

### Server Process

In [4]:
# For this Server function, the server listens to my local machine at Port 12345
# This is done because I do not have access to another PYNQ board
# So we will simulate another PYNQ board this way
# Input from simulated PYNQ board will be done by sending messages and decoding string for specific Words/Phrases

def server_p(i, _time):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # Bind the socket to the pynq board <CLIENT-IP> at port <LISTENING-PORT>
    sock.bind(('0.0.0.0', 12345))
    sock.listen()
    print('Waiting for Connection')
    # Accept connections
    conn, addr = sock.accept()
    # Receive bytes from the connection
    with conn:
        print(f"Connected by {addr} and listening") #Connected Message
        # Always running listening mode
        while True:
            data = conn.recv(1024)
            if data:
                if data.decode() == "Disconnecting": #if Client sent message saying Disconnecting
                    print(f"Communication Ended")
                    sock.close()                     #socket is closed to end connecting
                    break
                elif data.decode() == "Setting Alarm":   #if client sent message saying Setting Alarm
                    setAlarm(120)                       # set off alarm with Freq 120, assigned to simulated PYNQ
                else:
                    print(data.decode())


### Client Process

In [5]:
# For this Client function, the server connects to my local machine at Port 12345
# This is done because I do not have access to another PYNQ board, so my local machine is Simulating another PYNQ
# All button input for this Client code is done on the PYNQ board, no work around needed to fulfill requirement

cond = True
isConnected = False

def client_p(i,_time):
    global cond,isConnected
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    while cond:
        if base.buttons[0].read() != 0 and isConnected == False: #make sure not to connect more than once
            time.sleep(1)
            sock.connect(('192.168.2.1',65432))
            print('Connect to Server')
            isConnected = True
        while isConnected:
            if base.buttons[1].read() != 0:
                print('Button[1] Pressed: Turning on Server Alarm')
                sock.sendall(b'Turning on Alarm')
                time.sleep(.5)
                setAlarm(75)
            elif base.buttons[3].read() != 0:
                print('Button[3] pressed: Disconnecting from Server')
                sock.sendall(b'Disconnecting')
                time.sleep(1)
                sock.shutdown(1)
                sock.close()
                isConnected = False
                cond = False

            

#### Run 2 Processes

In [6]:
procsList = [] # list of 2 processes

# Launch process1 on CPU0
proc1_start = time.time()
proc1 = multiprocessing.Process(target=server_p, args=(0,proc1_start))
os.system("taskset -p -c [] []".format(0,proc1.pid))
proc1.start() #start process 1 for server code
procsList.append(proc1)



proc1.join()

# Launch process2 on CPU1
proc2_start = time.time()
proc2 = multiprocessing.Process(target=client_p, args=(1,proc2_start))
os.system("taskset -p -c [] []".format(1,proc2.pid))
proc2.start() #start process 1 for server code
procsList.append(proc2)
proc2.join()


Waiting for Connection
Connected by ('192.168.2.1', 60037) and listening
Disconnect
Communication Ended
Connect to Server
Button[1] Pressed: Turning on Server Alarm
Button[1] Pressed: Turning on Server Alarm
Button[3] pressed: Disconnecting from Server


In [ ]:
# The following is the Client code used to Simulate another PYNQ board on my local machine
# There are slight differences, since the PYNQ board is not used
# echo-client3.py

import socket

cond = True
isConnected = False

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('192.168.2.99',12345))
print('Connect to Server')
isConnected = True
while isConnected:
    getInput = input("Message to send:\n")
    msg = getInput.encode('utf-8')
    sock.sendall(msg)
    if msg.decode() == "Disconnecting":
        isConnected = False
        sock.shutdown(1)
        sock.close()
        break
            

In [ ]:
# The following is the Server code used to Simulate another PYNQ board on my local machine
# There are slight differences, since the PYNQ board is not used
# echo-server.py

import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# TODO:
# 1: Bind the socket to the pynq board <CLIENT-IP> at port <LISTENING-PORT>
sock.bind(('0.0.0.0', 65432))
# 2: Accept connections
sock.listen()
print('Waiting for Connection')
conn, addr = sock.accept()
# 3: Receive bytes from the connection
with conn:
    print(f"Connected by {addr} and listening")
    while True:
        data = conn.recv(1024)
        if data:
            if data.decode() == "Disconnecting":
                print(f"Communication Ended")
                break
            else:
                print(data.decode())

### Conclusion

My workflow throughout the assignment goes as follows:  

I first broke up the assignment into smaller functional areas (connecting the buzzer, generating a tone, button input, the server process, client process, and running 2 processes).  Since I do not have another PYNQ board to connect to, I had to think of a creative way to simulate the 2 processes on another board.  Once I got the buzzer working, I created an async function to set up and test the buzzer and connection requirements with the press of different buttons.  Once I got that working, I completed my server process and client process.

For the second PYNQ board simulation, I decided to run server/client code on my local machine and connect to the 2 processes running on the PYNQ board by using different Ports. (I am connected via ethernet)  I used Port 12345 for 1 set of client/server and 65432 for the other set of client/server.  To simulate button input from my local machine, I decided to send messages to the PYNQ board and have the Server code decode the message.  If I sent the message "Disconnecting" from my local machine, the PYNQ board would also disconnect.  When I sent the message "Turning on Alarm" this would indicate the PYNQ board to set the buzzer off.

An issue I came across was using the async function for button input.  I realized the processes were not sharing variables and it became a hassle to set flags for the client and server processes to read.  I realized later that I could directly check for button input within my server/client code and did not need the async function.  On the bright side, the button input function had helped me test my different functions.

To test my overall design, I tested one processes at a time.  This helped me debug the code on my local machine and get the correct communication betweent the PYNQ board and my simulated PYNQ board.  